# Building your deep neural network: Step by Step


Take home:
1. use non-linear units like ReLu to improve your model
2. Build a deeper neural network(with more than 1 hidden layer)
3. Implement an easy-to-use neural network class

# 1 Packages

* dnn_utils provides some necessary functions for this notebook
* testCase provides some test cases to assess the correctness of your functions
* np.random.seed(1) is used to keep all the random function calls consisten.

In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v3 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

# Outline of the Assignment

You will be implementing several "helper functions".
* Initialize the parameters for a two-layer netword and for an L-layer neural network
* Implement the forward propagation module
    * Complete the Linear part of a layer's forward propagation step
    * We give you the Activation function(relu/sigmoid)
    * Combine the previous two steps into a new linear-> activation forward cuntion
    

Note: for every forward function, there is a corresponding backward function. That's why we need to store some values in a cache. The cached values are useful for computing gradients.


# Initalization

two helper functions that will initialize the parameters for your model. The first function will be used to initalize parameters for a two layer model, the second will generalize this initialization process to L layers.

# 3.1 - 2-layer Neural Network

Exercise: Create and initialize the parameters of the 2-layer neural network

Instructions:
* The model's structure is: LINEAR -> RELU -> LINEAR -> SIGMOID
* Use random initialization for the weight matrices. Use np.random(shape)*0.01 with the corerect shape
* Use zero initialization for the biases. Use np.zeros(shape)

In [3]:
# Initialize parameters
def initialize_parameters(n_x, n_h, n_y):
    """
    n_x -- size of the input layer 
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
    W1 -- weight matrix of shape (n_h, n_x)
    b1 -- bias vector of shape (n_h, 1)
    W2 -- weight matrix of shape (n_y, n_h)
    b2 -- bias vector of shape (n_y, 1)
    
    """
    
    np.random.seed(1)
    
    W1 = np.random.randn(n_h, n_x) * 0.01 # X dimension is n_x, n_h
    b1 = np.zeros(shape = (n_h, 1)) # b1 dimension n_h
    W2 = np.random.randn(n_y, n_h) * 0.01 # correspond to (n_h, n_h)
    b2 = np.zeros(shape = (n_y, 1)) 
    
    assert(W1.shape == (n_h, n_x)) 
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h)) 
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,  #construct a dictionary struct to store parameters
                 "b1": b1,
                 "W2": W2,
                 "b2": b2}
    return parameters

In [4]:
parameters = initialize_parameters(2, 2, 1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))


W1 = [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1 = [[ 0.]
 [ 0.]]
W2 = [[ 0.00865408 -0.02301539]]
b2 = [[ 0.]]


3.2 L-layer Neural Network

In [5]:
# Initialize parameters deep
def initialize_parameters_deep(layer_dims):
    
    """
    Arguments:
    layer_dims -- python array (list) containing dimensions of each layer in our network
    
    Returns:
    parameters -- python dic struct
                    W1 -- weight matrix of shape(layer_dims[l], layer_dims[l - 1]
                    b1 -- bias vector of shape (layer_dims[])
    """
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the netword
    
    for l in range(1, L):
        # 
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], 
                                                   layer_dims[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], 
                                                  layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
        
    return parameters



In [6]:
parameters = initialize_parameters_deep([5, 4, 3])
print("W1 = " + str(parameters["W1"]) )
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[ 0.]
 [ 0.]
 [ 0.]]


# Forward propagation module

## Linear forward

forward propagation module.
Three functions in order:
* Linear
* Linear -> Activation where Activation will be eithre ReLU or sigmoid
* [Linear-> Relu] x (L - 1) -> Linear -> Sigmoid(whole model)


The linear forward module(vectorized over all examples) computes the following equations:

    Z[l] = W[l]A[l-1] + b[l]
    where A[0] = X
   

In [7]:
def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    
    assert(Z.shape == W.shape[0], A.shape[1])
    cache = (A, W, b)
    return Z, cache

In [8]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


## 4.2 Linear-Activation Forward



In [9]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->Activation layer
    
    Arguments:
    A_prev -- activations from previous layer(or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape(size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, l)
    
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu

    Returns:
    A -- the output of the activation function, also called the post-activation value
    cache -- a python dictionary contatining "linear_chace" and "activation_cache"
          -- stored for computing the backward pass efficiently
    """
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        
    elif activation == 'relu':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    assert(A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [10]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A  = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With Relu: A "+ str(A))

With sigmoid: A  = [[ 0.96890023  0.11013289]]
With Relu: A [[ 3.43896131  0.        ]]


## d) L-layer Model

For even more convenience when implementing L-layer Neural Net, you will need a function that replicates the previous one(linear_activation_forward with RELU) L - 1 times, then follows that with one linear_activation_forward with sigmoid



In [11]:
def L_model_forward(X, parameters):
    
    """
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize ( W b struct)
    
    """
    
    
    caches = []
    A = X
    L = len(parameters) // 2  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU] * (L -1).
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)],
                                            parameters['b' + str(1)],
                                            activation = 'relu')
        caches.append(cache)
        
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)],
                                         parameters['b' + str(L)],
                                         activation = 'sigmoid')
    caches.append(cache)
    
    assert(AL.shape == (1, X.shape[l]))
    
    return AL, caches

In [12]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[ 0.17007265  0.2524272 ]]
Length of caches list = 2


## Cost function

In [13]:
def compute_cost(AL, Y):
    """
    Arguments: 
    AL -- probability vector corresonding to your label predictions, shape
    (1, number of examples)
    
    Y -- true "label" vector -> output
    
    Returns:
    cost -- cross-entropy cost
    
    """
    m = Y.shape[1]
    
    # by formula
    cost = (-1 / m) * np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1 - Y, np.log(1 - AL)))
    
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    return cost

In [14]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.414931599615


# Backward prop

* LINEAR backward
* LINEAR -> ACTIVATION backward where ACTIVATION computes the derivative of either of the ReLU or sigmoid function
* LINEAR -> RELU x (L - 1) -> LINEAR -> SIGMOID backward (whole model)

In [15]:
def linear_backward(dZ, cache):
    
    """
    Arguments:
    dZ -- Gradients of the cost with repect to linear output
    cache -- tuple of values (A_prev, W, b)
    
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = np.dot(dZ, cache[0].T)/m
    db = np.squeeze(np.sum(dZ, axis = 1, keepdims = True)) / m
    dA_prev = np.dot(cache[1].T, dZ)
    
    assert(dA_prev.shape == A_prev.shape)
    assert(dW.shape == W.shape)
    assert(isinstance(db, float))
    
    return dA_prev, dW, db

In [16]:
# set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)

print("dA_prev  = " + str(dA_prev))
print("dW = " + str(dW))
print("db = " + str(db))

dA_prev  = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = 0.506294475007


## 6.2 Linear-Activation backward

* Sigmoid_backward: Implements the backward propagation for SIGMOID unit.

dZ = sigmoid_backward(dA, activation_cache)

* Relu_backward: Implements the backward propagation for RELU unit.

dZ = relu_backward(dA, activation_cache) 

In [17]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
        
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db


In [18]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, 
                                            activation = "sigmoid")
print("sigmoid:")
print("dA_prev = " + str(dA_prev))
print("dW = " + str(dW))
print("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, 
                                            activation = "relu")
print("relu:")
print("dA_prev = " + str(dA_prev))
print("dW = " + str(dW))
print("db = " + str(db) + "\n")


sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = -0.0572962221763

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = -0.208378923703




## L-model Backward

In [24]:
def L_model_backward(AL, Y, caches): 
    """
    Basically, backward prop: [LINEAR->RELU] * (L - 1) -> LINEAR -> SIGMOID
    
    Arguments:
    AL -> probability vector, output of the forward propagation
    Y --> True "label" vecotr
    caches -- list of caches contatining:
                * Cache of linear_activation_forward with "relu"
                * cache of linear_activation_forward with "sigmoid"
    Returns:
    grads -- A dictionary with the gradients
            grads["dA" + str(1)] = ...
            grads["dW" + str(l)] = ...
            grads["db" + str(l)] = ...
    
    """
    
    grads = {}
    L = len(caches); #layers
    m = AL.shape[1]; 
    Y = Y.reshape(AL.shape) 
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    current_cache = caches[-1]
    
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_backward(sigmoid_backward(dAL,
                                                                                                       current_cache[1]),
                                                                                      current_cache[0])
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients
        # Inputs: "grads["dA" + str(1 + 2)], caches"
        
        current_cache = cache[l]
        dA_prev_temp, dW_temp, db_temp = linear_backward(sigmoid_backward(dAL, current_cache[1]),
                                                        current_cache[0])
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        
    return grads

    
    

In [ ]:
X_assess, Y_assess, AL, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

## Update parameters


In [26]:
def update_parameters(parameters, grads, learning_rate):
    """
    Arguments:
    parameters -- python dictionary 
    grads -- python dictionary 
    
    Returns:
    parameters -- python dictionary 
    parameters["W1"] = ....etc
    
    """
    L = len(parameters)
    for l in range(L):
    
        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]

        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]
        
    return parameters
        
        

In [ ]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = " + str(parameters["W1"]))
print ("b1 = " + str(parameters["b1"]))
print ("W2 = " + str(parameters["W2"]))
print ("b2 = " + str(parameters["b2"]))
print ("W3 = " + str(parameters["W3"]))
print ("b3 = " + str(parameters["b3"]))